In [1]:
import json2frames as jf

# video_path = "../Videos/PL0138/"
video_path = "./videos/"
target_path = "./imagens_PL0138/"
json_file = './json_files/PL0138/annotations.json'

jf.get_frames_per_tag(video_path, json_file, target_path, square=True)

  0%|          | 0/12 [00:00<?, ?it/s]

Directory './imagens_PL0138/' already exists or invalid path

Directory './imagens_PL0138/duto' already exists or invalid path



  8%|▊         | 1/12 [05:45<1:03:22, 345.69s/it]

Bad time annotation on video 'HD_C_pl0138_VOL007'
{'startTime': '00:17:21', 'endTime': '00:17:23    ', 'tags': ['dano-duvida']}
endTime not in a correct time format


 33%|███▎      | 4/12 [2:18:12<4:36:24, 2073.12s/it]

There are no annotations for video HD_C_pl0138_VOL017.mp4


100%|██████████| 12/12 [3:01:42<00:00, 908.51s/it]  

The conversion took 03:01:42


In [2]:
import json2frames as jf

video_path = "../Videos/PL0138/"
target_path = "./imagens_PL0138/"
json_file = './json_files/PL0138/annotations17.json'
#tags estao no commons

jf.get_frames_per_tag(video_path, json_file, target_path, offset=-0.9, square=True)

  0%|          | 0/12 [00:00<?, ?it/s]

Directory './imagens_PL0138/' already exists or invalid path

Directory './imagens_PL0138/duto' already exists or invalid path



 25%|██▌       | 3/12 [00:00<00:01,  7.60it/s]

There are no annotations for video HD_C_pl0138_VOL005.mp4
There are no annotations for video HD_C_pl0138_VOL007.mp4
There are no annotations for video HD_C_pl0138_VOL006.mp4


There are no annotations for video HD_C_pl0138_VOL003.mp4


AttributeError: 'NoneType' object has no attribute 'shape'

In [2]:
from json2frames import *
import os 
import json
import cv2
import numpy as np
from time import time
from tqdm import tqdm
from commons import *

In [ ]:
def generate_frames(beginings_str, beginings_sec, ends_str, ends_sec, tag, frame_rate, auto_tag):
    for i,t in enumerate(beginings_sec):
        if (ends_sec[i] - beginings_sec[i] > 0):
            for j,frame_time in enumerate(np.linspace(beginings_sec[i],
                                                      ends_sec[i],
                                                      frame_rate*(ends_sec[i] - beginings_sec[i]),
                                                      endpoint=False)):
                video.set(cv2.CAP_PROP_POS_MSEC,frame_time*1000)
                errRead, frame = video.read()
                target_name = video_name+'_'+tag+'_'+beginings_str[i].replace(":","-")+"_"+str(j)+'.jpg'
                errWrite = cv2.imwrite(os.path.join(target_path,tag,target_name), frame)
        else:
            if not auto_tag:
                print("Bad time annotation on video '"+video_name)
                print("'{startTime: '"+beginings_str[i]
                      +"', 'endTime': '"+ends_str[i]
                      +"', 'tags': ['"+tag+"']}")
                print("startTime >= endTime")

In [ ]:
# video_name = "HD_C_pl0138_VOL009"
# with open('annot_test_auto_duct.json') as f:
#     jason = json.load(f)
def get_duct_times(json_data, video_name, event_tags, confusion_time=1):

    video_end = [video["videoEndTime"] for video in json_data["videos"] if video["videoName"] == video_name][0]
    beginings_sec = []
    ends_sec = []

    for tag in event_tags:
        beg_str, beg_sec, end_str, end_sec = get_times(json_data,tag,video_name)
        beginings_sec.extend(beg_sec)
        ends_sec.extend(end_sec)

    beginings_sec = sorted(beginings_sec)
    ends_sec = sorted(ends_sec)

    beginings_sec = [begining-confusion_time for begining in beginings_sec]
    ends_sec = [end+confusion_time for end in ends_sec]

    duct_beginings_sec = ends_sec
    duct_ends_sec = beginings_sec

    duct_beginings_sec.insert(0,0)
    duct_ends_sec.append(string2seconds(video_end))
    duct_beginings_str = [seconds2string(sec) for sec in duct_beginings_sec]
    duct_ends_str = [seconds2string(sec) for sec in duct_ends_sec]
    
    return duct_beginings_str, duct_beginings_sec, duct_ends_str, duct_ends_sec

# print(duct_beginings_sec)
# print(duct_ends_sec)
# print(duct_beginings_str)
# print(duct_ends_str)

In [ ]:
video_path = "../Videos"
target_path = "./imagens"
# json_file = 'json_bak/annot_test.json'
json_file = 'annot_test_auto_duct.json'
frame_rate = 3
#tags estao no commons
automatic_fill_duct = True
confusion_time = 1

with open(json_file) as f:
    jason = json.load(f)
video_list = [i for i in os.listdir(video_path) if i.endswith(".mp4")]
begin = time()

mkdir(target_path)
for tag in tags:    
    mkdir(os.path.join(target_path,tag))

annotated_video_list=[video["videoName"]+".mp4" for video in jason["videos"]]
for video_full_name in tqdm(video_list):
    video_name = video_full_name.split(sep=".")[0]
    video = cv2.VideoCapture(os.path.join(video_path,video_full_name))
    video_found = False
    for annotated_video in annotated_video_list:
        if(video_full_name == annotated_video):
            video_found = True
            for tag in tags:
                begin_string, begin_seconds, end_string, end_seconds = get_times(jason,tag,video_name)
                automatic_tag = False
                generate_frames(begin_string,begin_seconds,end_string,end_seconds,tag,frame_rate,automatic_tag)

            if automatic_fill_duct:
                automatic_tag = True
                begin_str,begin_sec,end_str,end_sec = get_duct_times(jason, video_name,alteration_tags)
                generate_frames(begin_str, begin_sec, end_str, end_sec, clean_duct_tag, clean_duct_frame_rate, automatic_tag)

    if not video_found:
        print("There are no annotations for video "+video_full_name)

        
end = time()
print("The convertion took "+seconds2string(round(end-begin)))

In [ ]:
video_name

In [ ]:
# duct_beginings_sec
# duct_ends_sec
# duct_beginings_str
# duct_ends_str
# target_path
tag = clean_duct_tag
# target_name
# video_name
# frame_rate = 1

for i,t in enumerate(duct_beginings_sec):
    if (duct_ends_sec[i] - duct_beginings_sec[i] > 0):
        for j,frame_time in enumerate(np.linspace(duct_beginings_sec[i],
                                                  duct_ends_sec[i],
                                                  clean_duct_frame_rate*(duct_ends_sec[i] - duct_beginings_sec[i]),
                                                  endpoint=False)):
            video.set(cv2.CAP_PROP_POS_MSEC,frame_time*1000)
            errRead, frame = video.read()
            target_name = video_name+'_'+tag+'_'+duct_beginings_str[i].replace(":","-")+"_"+str(j)+'.jpg'
            errWrite = cv2.imwrite(os.path.join(target_path,tag,target_name), frame)
    else:
        if not auto_tag:
            print("Bad time annotation on video '"+video_name)
            print("'{startTime: '"+duct_beginings_str[i]
                  +"', 'endTime': '"+duct_ends_str[i]
                  +"', 'tags': ['"+tag+"']}")
            print("startTime >= endTime")

bak do codigo

In [ ]:
targetName

In [ ]:
get_times(jason,"variacao","HD_C_pl0138_VOL009")

In [ ]:
jason

In [ ]:
[i for i in os.listdir(video_path) if len(i.split(sep=".")) == 2]

In [ ]:
time="00:00:59"
time[3:5]

In [ ]:
time

In [ ]:
video_list = ["HD_C_pl0138_VOL003.mp4","HD_C_pl0138_VOL009.mp4"]
for vid in video_list:
    video_name = vid.split(sep=".")[0]
    print(vid)

In [ ]:
enumerate(np.linspace(tag_begin_seconds[0], tag_end_seconds[0],frame_rate, endpoint=False))

In [ ]:
tag="duto"
vid = "HD_C_pl0138_VOL003.mp4"

video_name = vid.split(sep=".")[0]
video = cv2.VideoCapture(os.path.join(video_path,vid))


tag_begin_string, tag_begin_seconds, tag_end_string, tag_end_seconds = jf.get_times(jason,tag,video_name)
#tenho que consertar se o endtime for menor que o starttime
for i,time in enumerate(tag_begin_seconds):
    for j,frame_time in enumerate(np.linspace(tag_begin_seconds[i], tag_end_seconds[i],frame_rate**(tag_end_seconds[i] - tag_begin_seconds[i]), endpoint=False)):
        video.set(cv2.CAP_PROP_POS_MSEC,frame_time*1000)
        errRead, frame = video.read()
        target_name = video_name+'_'+tag+'_'+tag_begin_string[i].replace(":","-")+"_"+str(j)+'.png'
        errWrite = cv2.imwrite(os.path.join(target_path,tag,target_name), frame)

In [ ]:
jf.get_times(jason,tag,video_name)

In [ ]:
np.linspace(tag_begin_seconds[0], tag_end_seconds[0],frame_rate**(tag_end_seconds[0] - tag_begin_seconds[0]), endpoint=False)

In [ ]:
np.linspace(0, 36,frame_rate*(tag_end_seconds[0] - tag_begin_seconds[0]), endpoint=False)

In [ ]:
tag_end_seconds[0]

In [ ]:
x=                {
                    "startTime": "00:03:08",
                    "tags": [ "dano-duvida" ]
                }
x["startTime"]

In [ ]:
from time import strptime
strptime("00:12", '%H:%M:%S')

In [3]:
frame_time = 10
video_path = '../Videos/PL0018/'
video_full_name = 'HD_C_pl0018_VOL005.mp4'
video_name = 'HD_C_pl0018_VOL005'

video = cv2.VideoCapture(os.path.join(video_path,video_full_name))
video.set(cv2.CAP_PROP_POS_MSEC,frame_time*1000)
errRead, frame = video.read()
target_name = video_name+str(1)+'.jpg'
border = int((frame.shape[1] - frame.shape[0])/2)
# frame = frame[:,border:frame.shape[1]-border,:]
frame.shape

# errWrite = cv2.imwrite(os.path.join(target_path,tag,target_name), frame)

(720, 1280, 3)